In [120]:
import pandas as pd
import matplotlib.pyplot as plt

In [121]:
#read csv that contains info for people in the industry
talent = pd.read_csv('zippedData/imdb.name.basics.csv.gz')

#import data from movies WITH $
movies = pd.read_csv('zippedData/movie_data.csv',index_col=[0])
#include and add ROI
movies['d_roi'] = movies['domestic_gross'] / movies['production_budget']

Merge talent dataframe with movies data frame to have a data set where we can see which people are associated with genres. 
- Explode known titles column so that each row represents a film and the person associated with it. 
- Explode profession column so that each person is associated with a single movie (for merging purposes)
- Explode genre so that there is only one genre on each line. 

Talent table: 

In [122]:
talent.columns

Index(['nconst', 'primary_name', 'birth_year', 'death_year',
       'primary_profession', 'known_for_titles'],
      dtype='object')

In [123]:
talent.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [124]:
talent['primary_profession']=talent['primary_profession'].str.split(",")

In [125]:
talent = talent.explode('primary_profession')

In [126]:
talent['known_for_titles']=talent['known_for_titles'].str.split(",")

In [127]:
talent = talent.explode('known_for_titles')

In [135]:
talent = talent.drop(columns = ['birth_year', 'death_year'])

KeyError: "['birth_year' 'death_year'] not found in axis"

In [136]:
talent = talent.rename(columns={"known_for_titles": "tconst"})

In [137]:
talent = talent.dropna()

In [138]:
talent.shape

(3434013, 4)

Movie genre

In [131]:
movies.columns

Index(['tconst', 'merge_id', 'release_date', 'genres', 'averagerating',
       'numvotes', 'production_budget', 'domestic_gross', 'worldwide_gross',
       'd_roi'],
      dtype='object')

In [132]:
movies['genres'] = movies['genres'].str.split(",")
movies = movies.explode('genres')

Join the data frames together

In [141]:
movie_data = talent.set_index('tconst').join(movies.set_index('tconst'))
movie_data

,nconst,primary_name,primary_profession,merge_id,release_date,genres,averagerating,numvotes,production_budget,domestic_gross,worldwide_gross,d_roi
tconst,,,,,,,,,,,,
tt0000014,nm0166380,François Clerc,actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000014,nm0244989,Benoît Duval,actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000038,nm2958402,Scott Stratton,actor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000038,nm2958402,Scott Stratton,writer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt0000038,nm2958402,Scott Stratton,composer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
tt9916754,nm9275320,Daniel Cardoso,animation_department,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt9916754,nm9275320,Daniel Cardoso,cinematographer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tt9916764,nm6687687,Meleisha Edwards,producer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [142]:
movie_data.dropna(subset=['averagerating'], inplace=True)

In [147]:
movie_data.shape

(270216, 12)

Find directors of documentaries with high ratings

In [154]:
doc_talent = movie_data[(movie_data['genres']=="Documentary") & \
                       

(10, 12)

In [37]:
#split professions to find directors and producers 
split = talent['primary_profession'].str.split(",", expand = True)
talent['one']= split[0]
talent['two']= split[1]
talent['three']= split[2]
g1 = talent[['nconst','primary_name', 'known_for_titles','one']].rename(columns={'one':'profession'})
g2 = talent[['nconst','primary_name', 'known_for_titles','two']].rename(columns={'two':'profession'})
g3 = talent[['nconst','primary_name', 'known_for_titles','three']].rename(columns={'three':'profession'})
jobs_list = [g1, g2, g3]
talent = pd.concat(jobs_list)
talent.dropna(inplace=True)
talent

,nconst,primary_name,known_for_titles,profession
0,nm0061671,Mary Ellen Bauder,"tt0837562,tt2398241,tt0844471,tt0118553",miscellaneous
1,nm0061865,Joseph Bauer,"tt0896534,tt6791238,tt0287072,tt1682940",composer
2,nm0062070,Bruce Baum,"tt1470654,tt0363631,tt0104030,tt0102898",miscellaneous
3,nm0062195,Axel Baumann,"tt0114371,tt2004304,tt1618448,tt1224387",camera_department
4,nm0062798,Pete Baxter,"tt0452644,tt0452692,tt3458030,tt2178256",production_designer
...,...,...,...,...
606545,nm9914242,Abu Iddris,"tt8574516,tt9490498",writer
606560,nm9928618,Roberto Farías,"tt1941928,tt3328948,tt1327798,tt4375438",writer
606581,nm9943777,Jeremy Bradshaw,"tt8474782,tt9597440,tt8865220,tt9133458",producer
606619,nm9971271,Ray Spivey,tt8697454,director


In [ ]:
#split known titles to find the titles each person is known for
split = talent['known_for_titles'].str.split(",", expand = True)
talent['one']= split[0]
talent['two']= split[1]
talent['three']= split[2]
talent['four']= split[2]
g1 = talent[['nconst','primary_name', 'profession','one']].rename(columns={'one':'profession'})
g2 = talent[['nconst','primary_name', 'known_for_titles','two']].rename(columns={'two':'profession'})
g3 = talent[['nconst','primary_name', 'known_for_titles','three']].rename(columns={'three':'profession'})
jobs_list = [g1, g2, g3]
talent = pd.concat(jobs_list)
talent.dropna(inplace=True)
talent

In [4]:
#investigate profession
talent['primary_profession'].sample(10)

194314                                            actress
498033                               writer,miscellaneous
193290                           producer,writer,director
543742    cinematographer,camera_department,miscellaneous
556205        director,cinematographer,assistant_director
331252                           editor,producer,director
165872                          producer,director,actress
424145                                    cinematographer
327480                                         soundtrack
457430                                              actor
Name: primary_profession, dtype: object

In [5]:
#import data from movies WITH $
movies = pd.read_csv('zippedData/movie_data.csv',index_col=[0])
movies['d_roi'] = movies['domestic_gross'] / movies['production_budget']

In [6]:
movies.shape

(1371, 10)

In [7]:
# split = table
# split = split[column].str.split(",", expand = True)
# split['one']= split[0]
# split['two']= split[1]
# split['three']= split[2]
# split['four']= split[3]
# columns = list(category.columns)
# l1 = genre[[columns, 'one']].rename(columns={'one':column})
# l2 = genre[[columns, 'two']].rename(columns={'one':column})
# l3 = genre[[columns, 'three']].rename(columns={'one':column})
# l4 = genre[[columns, 'four']].rename(columns={'one':column})
# l2 = genre[['tconst','averagerating', 'numvotes', 'production_budget', 'domestic_gross', 'worldwide_gross',"d_roi", 'two']].rename(columns={'two':'genre'})
# l3 = genre[['tconst','averagerating', 'numvotes', 'production_budget', 'domestic_gross', 'worldwide_gross',"d_roi", 'three']].rename(columns={'three':'genre'})
# l4
# genres_list = [g1, g2, g3]
# genres = pd.concat(genres_list)
# genres.dropna(inplace=True)
# genres.head()

In [8]:
#find top documentaries by ratings
doc = pd.read_csv('zippedData/imdb_merge.csv',index_col=[0])
doc

,tconst,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,merge_id
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,2013 Sunghursh
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,2019 One Day Before the Rainy Season
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517,2018 The Other Side of the Wind
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13,2018 Sabse Bada Sukh
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119,2017 The Wandering Soap Opera
...,...,...,...,...,...,...,...,...,...
73849,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130.0,Drama,8.4,365,2019 Padmavyuhathile Abhimanyu
73850,tt9913056,Swarm Season,Swarm Season,2019,86.0,Documentary,6.2,5,2019 Swarm Season
73851,tt9913084,Diabolik sono io,Diabolik sono io,2019,75.0,Documentary,6.2,6,2019 Diabolik sono io
73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,98.0,"Drama,Family",8.7,136,2019 Sokagin Çocuklari


In [9]:
#separating genres
genre = doc
split = genre['genres'].str.split(",", expand = True)
genre['one']= split[0]
genre['two']= split[1]
genre['three']= split[2]
g1 = genre[['tconst','averagerating', 'numvotes', 'start_year','one']].rename(columns={'one':'genre'})
g2 = genre[['tconst','averagerating', 'numvotes', 'start_year','two']].rename(columns={'two':'genre'})
g3 = genre[['tconst','averagerating', 'numvotes', 'start_year','three']].rename(columns={'three':'genre'})
genres_list = [g1, g2, g3]
doc = pd.concat(genres_list)
doc.dropna(inplace=True)
doc

,tconst,averagerating,numvotes,start_year,genre
0,tt0063540,7.0,77,2013,Action
1,tt0066787,7.2,43,2019,Biography
2,tt0069049,6.9,4517,2018,Drama
3,tt0069204,6.1,13,2018,Comedy
4,tt0100275,6.5,119,2017,Comedy
...,...,...,...,...,...
73819,tt9889926,5.2,13,2018,Romance
73820,tt9889952,8.0,12,2018,Romance
73821,tt9890030,7.4,13,2017,Romance
73827,tt9896916,7.9,61,2019,Family


In [10]:
# #dataframe of documentaries with their ratings
# doc= doc[doc['genre']=='Documentary']
# #dataframe of documentaries with their ratings
# scifi= doc[doc['genre']=='Sci-Fi']

In [11]:
#use talent dataframe to find people who worked on movies 
#in the doc dataframe
talent.shape

(606648, 6)

In [12]:
#split known titles to their own rows so there is only one tconst on a row
doc_talent = talent
split = talent['known_for_titles'].str.split(",", expand = True)
talent['one']= split[0]
talent['two']= split[1]
talent['three']= split[2]
talent['four']= split[3]
g1 = talent[['nconst','primary_name', 'primary_profession', 'one']].rename(columns={'one':'tconst'})
g2 = talent[['nconst','primary_name', 'primary_profession', 'two']].rename(columns={'two':'tconst'})
g3 = talent[['nconst','primary_name', 'primary_profession', 'three']].rename(columns={'three':'tconst'})
g4 = talent[['nconst','primary_name', 'primary_profession', 'four']].rename(columns={'four':'tconst'})
genres_list = [g1, g2, g3, g4]
doc_talent = pd.concat(genres_list)
doc_talent.dropna(inplace=True)
doc_talent

,nconst,primary_name,primary_profession,tconst
0,nm0061671,Mary Ellen Bauder,"miscellaneous,production_manager,producer",tt0837562
1,nm0061865,Joseph Bauer,"composer,music_department,sound_department",tt0896534
2,nm0062070,Bruce Baum,"miscellaneous,actor,writer",tt1470654
3,nm0062195,Axel Baumann,"camera_department,cinematographer,art_department",tt0114371
4,nm0062798,Pete Baxter,"production_designer,art_department,set_decorator",tt0452644
...,...,...,...,...
606579,nm9942718,Witri Asmaida,actress,tt8638476
606581,nm9943777,Jeremy Bradshaw,"actor,cinematographer,producer",tt9133458
606593,nm9951829,Hsin-Yen Hsu,actress,tt6270494
606628,nm9980896,Rama Narayanan,"director,writer",tt8929818


In [13]:
#join doc and doc talent 
docs_joined = doc_talent.set_index('tconst').join(doc.set_index('tconst'))

In [14]:
docs_joined.dropna(subset=['averagerating'],inplace=True)

In [15]:
docs_joined

,nconst,primary_name,primary_profession,averagerating,numvotes,start_year,genre
tconst,,,,,,,
tt0063540,nm1194313,Mahasweta Devi,writer,7.0,77.0,2013.0,Action
tt0063540,nm1194313,Mahasweta Devi,writer,7.0,77.0,2013.0,Crime
tt0063540,nm1194313,Mahasweta Devi,writer,7.0,77.0,2013.0,Drama
tt0063540,nm1391276,Anjana Rawail,"writer,costume_designer",7.0,77.0,2013.0,Action
tt0063540,nm1391276,Anjana Rawail,"writer,costume_designer",7.0,77.0,2013.0,Crime
...,...,...,...,...,...,...,...
tt9913056,nm1502645,Sarah Christman,"miscellaneous,director,editor",6.2,5.0,2019.0,Documentary
tt9914286,nm1902682,Kasim Uçkan,writer,8.7,136.0,2019.0,Drama
tt9914286,nm1902682,Kasim Uçkan,writer,8.7,136.0,2019.0,Family


In [16]:
docs_joined[docs_joined['genre']=='Documentary'].describe()

,averagerating,numvotes,start_year
count,56994.000000,56994.000000,56994.000000
mean,7.408783,527.672071,2014.096028
std,1.039426,3126.980653,2.487205
min,1.000000,5.000000,2010.000000
25%,6.800000,10.000000,2012.000000
50%,7.400000,25.000000,2014.000000
75%,8.100000,110.000000,2016.000000
max,10.000000,74978.000000,2019.000000


In [17]:
#sort the list for the top documentaries (>75%)
top_docs = docs_joined[(docs_joined['averagerating']>8.1) & (docs_joined['genre']=='Documentary')]

In [18]:
#split professions to find directors and producers 
split = top_docs['primary_profession'].str.split(",", expand = True)
top_docs['one']= split[0]
top_docs['two']= split[1]
top_docs['three']= split[2]
g1 = top_docs[['nconst','primary_name', 'averagerating', 'start_year','one']].rename(columns={'one':'profession'})
g2 = top_docs[['nconst','primary_name', 'averagerating', 'start_year','two']].rename(columns={'two':'profession'})
g3 = top_docs[['nconst','primary_name', 'averagerating', 'start_year','three']].rename(columns={'three':'profession'})
jobs_list = [g1, g2, g3]
top_docs = pd.concat(jobs_list)
top_docs.dropna(inplace=True)
top_docs

<ipython-input-18-c8d2f842918e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_docs['one']= split[0]
<ipython-input-18-c8d2f842918e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_docs['two']= split[1]
<ipython-input-18-c8d2f842918e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,nconst,primary_name,averagerating,start_year,profession
tconst,,,,,
tt0430524,nm1749018,Ermanno Vanino,8.7,2011.0,cinematographer
tt0436782,nm0074206,Howard S. Berger,9.1,2013.0,director
tt0436782,nm5362951,Joseph L. Bagamery,9.1,2013.0,producer
tt0436782,nm5362958,Nora Moss,9.1,2013.0,producer
tt0436782,nm1167800,Susan Stahman,9.1,2013.0,producer
...,...,...,...,...,...
tt9899290,nm3305827,Jason S. Berlin,8.7,2019.0,producer
tt9899290,nm2243354,Adam Avagliano,8.7,2019.0,miscellaneous
tt9904844,nm4256971,Israel Bañuelos,8.7,2019.0,cinematographer


In [19]:
#create a list of directors from recent years with top ratings
top_docs = top_docs[(top_docs['profession']=='director') & \
                    (top_docs['averagerating'] > 9.7) & \
                   (top_docs['start_year'] > 2016)\
                   ]

In [20]:
top_docs.sort_values(by='averagerating',ascending=False)

,nconst,primary_name,averagerating,start_year,profession
tconst,,,,,
tt7227500,nm1249950,Robert Tutak,10.0,2018.0,director
tt8730716,nm4568586,Taylor Morden,10.0,2019.0,director
tt5089804,nm7633303,Lindsay Thompson,10.0,2019.0,director
tt7227500,nm5335596,Saud Jubaer,10.0,2018.0,director
tt9743544,nm10174241,Andrew Jezard,9.9,2018.0,director
tt9747294,nm3441491,Jesse Gilbert,9.9,2019.0,director
tt4023464,nm6757200,Nick Saporito,9.8,2018.0,director
tt4023464,nm8459774,Jonathan Turcotte,9.8,2018.0,director
tt4818518,nm0359087,Lynn Hamrick,9.8,2018.0,director


In [21]:
#create a list of directors from recent years with top ratings
top_docs_misc = top_docs[(top_docs['averagerating'] > 9.7) & \
                   (top_docs['start_year'] > 2016)\
                   ]
top_docs_misc.sort_values(by='averagerating', ascending=False).head(10)

,nconst,primary_name,averagerating,start_year,profession
tconst,,,,,
tt7227500,nm1249950,Robert Tutak,10.0,2018.0,director
tt8730716,nm4568586,Taylor Morden,10.0,2019.0,director
tt5089804,nm7633303,Lindsay Thompson,10.0,2019.0,director
tt7227500,nm5335596,Saud Jubaer,10.0,2018.0,director
tt9743544,nm10174241,Andrew Jezard,9.9,2018.0,director
tt9747294,nm3441491,Jesse Gilbert,9.9,2019.0,director
tt4023464,nm6757200,Nick Saporito,9.8,2018.0,director
tt4023464,nm8459774,Jonathan Turcotte,9.8,2018.0,director
tt4818518,nm0359087,Lynn Hamrick,9.8,2018.0,director


# scifi

In [22]:
docs_joined

,nconst,primary_name,primary_profession,averagerating,numvotes,start_year,genre
tconst,,,,,,,
tt0063540,nm1194313,Mahasweta Devi,writer,7.0,77.0,2013.0,Action
tt0063540,nm1194313,Mahasweta Devi,writer,7.0,77.0,2013.0,Crime
tt0063540,nm1194313,Mahasweta Devi,writer,7.0,77.0,2013.0,Drama
tt0063540,nm1391276,Anjana Rawail,"writer,costume_designer",7.0,77.0,2013.0,Action
tt0063540,nm1391276,Anjana Rawail,"writer,costume_designer",7.0,77.0,2013.0,Crime
...,...,...,...,...,...,...,...
tt9913056,nm1502645,Sarah Christman,"miscellaneous,director,editor",6.2,5.0,2019.0,Documentary
tt9914286,nm1902682,Kasim Uçkan,writer,8.7,136.0,2019.0,Drama
tt9914286,nm1902682,Kasim Uçkan,writer,8.7,136.0,2019.0,Family


In [23]:
#split professions to find directors and producers 
split = docs_joined['primary_profession'].str.split(",", expand = True)
docs_joined['one']= split[0]
docs_joined['two']= split[1]
docs_joined['three']= split[2]
g1 = docs_joined[['nconst','primary_name', 'averagerating', 'start_year','genre','one']].rename(columns={'one':'profession'})
g2 = docs_joined[['nconst','primary_name', 'averagerating', 'start_year','genre','two']].rename(columns={'two':'profession'})
g3 = docs_joined[['nconst','primary_name', 'averagerating', 'start_year','genre','three']].rename(columns={'three':'profession'})
jobs_list = [g1, g2, g3]
scifi = pd.concat(jobs_list)
scifi.dropna(inplace=True)
scifi

,nconst,primary_name,averagerating,start_year,genre,profession
tconst,,,,,,
tt0063540,nm1194313,Mahasweta Devi,7.0,2013.0,Action,writer
tt0063540,nm1194313,Mahasweta Devi,7.0,2013.0,Crime,writer
tt0063540,nm1194313,Mahasweta Devi,7.0,2013.0,Drama,writer
tt0063540,nm1391276,Anjana Rawail,7.0,2013.0,Action,writer
tt0063540,nm1391276,Anjana Rawail,7.0,2013.0,Crime,writer
...,...,...,...,...,...,...
tt9905476,nm10523880,Steve Gherebean,9.3,2019.0,Documentary,director
tt9905476,nm4983283,Edward Khoma,9.3,2019.0,Documentary,editor
tt9910502,nm5543922,Erhan Uslu,7.0,2019.0,Comedy,soundtrack


In [24]:
scifi['averagerating'].describe()

count    1.997375e+06
mean     6.270720e+00
std      1.396511e+00
min      1.000000e+00
25%      5.500000e+00
50%      6.400000e+00
75%      7.200000e+00
max      1.000000e+01
Name: averagerating, dtype: float64

In [25]:
scifi['profession'].value_counts()

actor                        313930
producer                     282659
writer                       218740
director                     179943
actress                      166775
miscellaneous                 98181
cinematographer               95424
camera_department             85452
editor                        83446
composer                      83089
soundtrack                    52171
music_department              49785
editorial_department          36834
assistant_director            36474
sound_department              29580
production_manager            27565
art_department                27292
production_designer           21206
visual_effects                18288
art_director                  14119
stunts                        13420
executive                      9416
casting_department             7421
location_management            6450
make_up_department             5917
casting_director               5785
animation_department           5488
special_effects             

In [26]:
bestscifi = scifi[(scifi['profession']=='director') & \
                  (scifi['genre']=='Sci-Fi') & \
                  (scifi['start_year'] > 2016)].sort_values(by='averagerating', ascending=False)
bestscifi.head(10)

,nconst,primary_name,averagerating,start_year,genre,profession
tconst,,,,,,
tt6819176,nm3266312,Carla Albuquerque,9.2,2017.0,Sci-Fi,director
tt6819176,nm3850038,André Madrini,9.2,2017.0,Sci-Fi,director
tt6819176,nm7664327,Beto Ribeiro,9.2,2017.0,Sci-Fi,director
tt5812442,nm2610141,Michael Samer,9.1,2017.0,Sci-Fi,director
tt8617064,nm3701636,Joshua Courtade,9.0,2018.0,Sci-Fi,director
tt3274690,nm3045659,Emily Beach,9.0,2018.0,Sci-Fi,director
tt3274690,nm4493736,Alex Ojeda,9.0,2018.0,Sci-Fi,director
tt3274690,nm1142297,Michelle Opitz,9.0,2018.0,Sci-Fi,director
tt3274690,nm2181488,Olicer J. Muñoz,9.0,2018.0,Sci-Fi,director


In [27]:
scifi.loc[scifi['averagerating']==10]

,nconst,primary_name,averagerating,start_year,genre,profession
tconst,,,,,,
tt1770682,nm3240612,George Hurd,10.0,2010.0,Crime,actor
tt1770682,nm3240612,George Hurd,10.0,2010.0,Documentary,actor
tt1770682,nm4169504,Daniel Alexander,10.0,2010.0,Crime,editor
tt1770682,nm4169504,Daniel Alexander,10.0,2010.0,Documentary,editor
tt1770682,nm4166961,Daniel Alexander,10.0,2010.0,Crime,writer
...,...,...,...,...,...,...
tt7259300,nm8791543,Emre Oran,10.0,2019.0,Comedy,writer
tt7259300,nm3622687,Ufuk Gokkaya,10.0,2019.0,Adventure,camera_department
tt7259300,nm3622687,Ufuk Gokkaya,10.0,2019.0,Comedy,camera_department


In [28]:
bestscifi = scifi[(scifi['genre']=='Sci-Fi') & \
                  (scifi['start_year'] > 2016)].sort_values(by='averagerating', ascending=False)
bestscifi.head(15)

,nconst,primary_name,averagerating,start_year,genre,profession
tconst,,,,,,
tt6819176,nm3850038,André Madrini,9.2,2017.0,Sci-Fi,director
tt6819176,nm7664327,Beto Ribeiro,9.2,2017.0,Sci-Fi,writer
tt6819176,nm8963473,Marcelo Jose Assayag Rocha,9.2,2017.0,Sci-Fi,cinematographer
tt6819176,nm0786370,Ricardo Severo,9.2,2017.0,Sci-Fi,composer
tt6819176,nm3850038,André Madrini,9.2,2017.0,Sci-Fi,actor
tt6819176,nm8250208,Raphael Gama,9.2,2017.0,Sci-Fi,actor
tt6819176,nm8946682,Murilo Cezar,9.2,2017.0,Sci-Fi,actor
tt6819176,nm3266312,Carla Albuquerque,9.2,2017.0,Sci-Fi,producer
tt6819176,nm3266312,Carla Albuquerque,9.2,2017.0,Sci-Fi,art_director


In [33]:
#split_column table function 
def split_column(table, splitcolumn):
    split = table
    split = split[splitcolumn].str.split(",", expand = True)
    split['one']= split[0]
    split['two']= split[1]
    split['three']= split[2]
    #split['four']= split[3]
    
    columns = table.columns
    
    l1 = genre[[columns, 'one']].rename(columns={'one':column})
    l2 = genre[[columns, 'two']].rename(columns={'one':column})
    l3 = genre[[columns, 'three']].rename(columns={'one':column})
    l4 = genre[[columns, 'four']].rename(columns={'one':column})
    
    split_list = [g1, g2, g3, l4]
    table = pd.concat(split_list)
    table.dropna(inplace=True)
    table

In [34]:
split_column(talent, 'primary_profession')

TypeError: unhashable type: 'Index'